# Retail Demo Store - Location Workshop

Welcome to the Retail Demo Store Location Workshop. In this module we're going to be seeing a very simple example of how Amazon Location Services can be used as the basis for location-sensitive marketing and transacting. By using location events to initiate user flows, we can catch shoppers when they are in the right place and time.

In this case, we illustrate the use case of reminders for users to pick up orders. Here we make use of Amazon Pinpoint Transactional Messaging to send these transactional messages. We might equally send reminder campaigns based on recent purchases by users and products available or in overstock in stores nearby. If we were to do that we might use Pinpoint campaigns set up to message users based on events that we propagate from Location Services to Pinpoint using a similar method to that described here.

*For a more in-depth example using Amazon Personalize to generate personalized offers, the use of "Campaigns" in Amazon Pinpoint to target users with unfinished purchases and incorporate Amazon Personalize output, and the maps functionality of Amazon Location Services to visualise user behaviour, see the Location Services demo on the UI - see "Retail Geofencing and Location-aware Personalization" in the demo guide for details of that functionality.*

Recommended Time: 1 Hour

# Manual setup

**IMPORTANT: Please read this section carefully**.

Before you can successfully run the code below there are a small number of setup steps to do:

1. Go to the retail store storefront that was created when you deployed the Retail Demo Store with CloudFormation and make a user. Ensure that you create a user with a valid email, that you use. **Take a note of the username that you create below:**

In [ ]:
cognito_username = 'daemon'  # CHANGE THIS TO YOUR USERNAME


2. Additionally ensure that when you go into Amazon Pinpoint, select a project, and Settings > Email, that the email channel is enabled and that you have a verified email address set up as default sender (this can be the same email address as you saved against the user).

Note: at the bottom of this notebook is code for cleaning up the resources that you create.

# Setup

To get started, we need to perform a bit of setup. Walk through each of the following steps to configure your environment to interact with the Amazon Location Service.

In [ ]:
# Warning! IAM permissions for this notebook are set up by default for the following names 
# - if you change them you may no longer have permissions to run the below operations.
resource_name = 'RetailDemoStoreLocationWorkshop'  # We name all our Location resources this
event_handler_function_name = 'LocationNotebookEventHandler' 
eventbridge_target_id = 'GeofenceEventHandlerTarger'
eventbridge_rule_name = 'GeofenceEventHandlerRule'

Configure the name of resources you are going to create:

### Import Dependencies and Setup Boto3 Python Clients

Throughout this workshop we will need access to some common libraries and clients for connecting to AWS services. We also have to retrieve Uid from a SageMaker notebook instance tag.

In [ ]:
# While Location is in beta, we need to grab its service description - this is up to date in boto3

! pip install botocore==1.19.38 boto3==1.16.38

In [ ]:
# Import Dependencies

import boto3
import json
import time
import botocore
import os
import requests
from datetime import datetime

from sagemaker import get_execution_role as sagemaker_get_execution_role
from matplotlib import pyplot as plt

# Setup Clients
servicediscovery = boto3.client('servicediscovery')
location = boto3.client('location')
awslambda = boto3.client('lambda')
events = boto3.client('events')

# Retrieve this notebook instance's role
sagemaker_role = sagemaker_get_execution_role()
print('Sagemaker role:', sagemaker_role)

# Get some data passed from CloudFormation
with open('/opt/ml/metadata/resource-metadata.json') as f:
    data = json.load(f)
    sagemaker_client = boto3.client('sagemaker')
    sagemakerResponce = sagemaker_client.list_tags(ResourceArn=data["ResourceArn"])
    for tag in sagemakerResponce["Tags"]:
        if tag['Key'] == 'Uid':
            Uid = tag['Value']
            print('Uid:', Uid)
        if tag['Key'] == 'UserPoolId':
            UserPoolId = tag['Value']
            print('UserPoolId:', UserPoolId)
        if tag['Key'] == 'PinpointAppId':
            PinpointAppId = tag['Value']
            print('PinpointAppId:', PinpointAppId)            


# Create Location resources

## Create a Location Map

While not necessary for the following steps, the following will create a map which you could in applications use to visualise geofences etc. that you create. Navigate to the [Location UI Console](https://console.aws.amazon.com/location/) to see the map, and check out the SDKs for other use options.

In [ ]:

create_map_response = location.create_map(
    MapName=resource_name,
    Configuration={
        'Style': 'VectorEsriNavigation'
    },
    Description='Retail Demo Store Sample',
    PricingPlan='RequestBasedUsage'
)
print(json.dumps(create_map_response, indent=4, default=str))

## Create a Location Geofence Collection

The Geofence Collection is used to fire events when users cross into the geofence - it can be used to start user processes such as product collection or send located marketing messages.

In [ ]:
create_geofence_collection_response = location.create_geofence_collection(
    CollectionName=resource_name,
    Description='Retail Demo Store Sample',
    PricingPlan='RequestBasedUsage'
)
geofence_collection_arn = create_geofence_collection_response['CollectionArn']
print(json.dumps(create_geofence_collection_response, indent=4, default=str))

## Create a Location Geofence

We can add to Geofence Collections multiple geofences. Each one defines a region such that if a user moves from outside the union of the regions to the inside or visa versa, an event will be fired so that we can make use of this geographic aspect of user behaviour.

The following geofence represents an area around at latitude/longitude 0,0 at a size of 1 degree.

In [ ]:
store_geofence = \
          [[-1, 1],
           [-1,-1],
           [ 1,-1],
           [ 1, 1],
           [-1, 1]]

In [ ]:
put_geofence_response = location.put_geofence(
    CollectionName=resource_name,
    Geometry={"Polygon": [store_geofence]},
    GeofenceId=resource_name
)

print(json.dumps(put_geofence_response, indent=4, default=str))

Note that using the "maps" API of Location, it would be possible to visualise this Geofence on a map, along with other visualisations.

## Create a tracker

A tracker is used to record the locations of any number of "devices" that can send updates about ther position to Location. You can imagine that Location may be sending position updates from retailer apps on their phone and so forth.

In [ ]:
create_tracker_response = location.create_tracker(
    TrackerName=resource_name,
    Description='Retail Demo Store Sample',
    PricingPlan='RequestBasedUsage'
)

print(json.dumps(create_tracker_response, indent=4, default=str))

## Associate tracker with geofence

By associating a tracker with a geofence collection, we add a link between the tracker and the geofence such that any devices in the tracker impinging the geofences in the geofence collection will fire an AWS EventBridge event.

In [ ]:
associate_tracker_consumer_response = location.associate_tracker_consumer(
    ConsumerArn=geofence_collection_arn,
    TrackerName=resource_name
)

print(json.dumps(associate_tracker_consumer_response, indent=4, default=str))

# Handle Location Geofence Events

As mentioned above, we have set up an EventBridge event to fire whenever a user comes near our fictional store. What kind of things might we do with such events? Here, we set up an AWS Lambda function to be called when such events fire, that checks whether a user has any orders that they have made and accordingly sends them an email telling them to come and pick it up in-store.

For your reference, we provide here typical structure of a Location event. The DeviceID is filled in when the tracker is updated with a device's location via the Location API (in our application we fill DeviceID with a Cognito User ID e.g. "daemon"). 

Here is a typical structure of a Location enter event (exit events are also possible): 

        {
            "version": "0",
            "id": "12345678-9abc-def0-1234-56789abcdef0",
            "detail-type": "Location Geofence Event",
            "source": "aws.geo",
            "account": "YOUR_ACCOUNT_ID",
            "time": "2020-11-23T14:30:33Z",
            "region": "us-east-1",
            "resources": [
                "arn:aws:geo:us-east-1:YOUR_ACCOUNT_ID:geofence-collection/COLLECTIONID",
                "arn:aws:geo:us-east-1:YOUR_ACCOUNT_ID:tracker/TRACKERID"
            ],
            "detail": {
                "EventType": "ENTER",
                "GeofenceId": "GEOFENCEID",
                "DeviceId": "FILL_THIS_IN: WE_USE_COGNITO_USER",
                "SampleTime": "2020-11-23T14:30:32.867Z",
                "Position": [-100,50]
            }
        }


## Write a Lambda handler

Here we write a Lambda function to handle an event of this sort. It is going to use the Amazon Pinpoint [Transactional API](https://docs.aws.amazon.com/pinpoint/latest/developerguide/send-messages-sdk.html) to send email messages to users who have waiting orders. We will hit the orders microservice to check if orders are available. The Lambda function expects the Pinpoint App ID (Project ID), the Cognito User Pool ID in environment variables. The Lambda function is going to live in a subdirectory called "location-event-handler".

In [ ]:
! [ -d location-event-handler ] && rm -r location-event-handler  # remove Lambda bundle if exists
! mkdir location-event-handler  # We will put Lambda function and dependencies in here

In the following function, we receive an event and send an email to the user configured at the top of this notebook.

We could also, for example, as an extra step, hit the orders service to retrieve order details.

The ``%%writefile`` directive writes a Jupyter notebook cell as a file - that way you can see here what we are going to send as an AWS Lambda function. In later cells we will deploy this file as an AWS Lambda function.

In [ ]:
%%writefile location-event-handler/location-event-handler.py

import boto3
import logging
import os 
import urllib3
import json

pinpoint = boto3.client('pinpoint')
cognito_idp = boto3.client('cognito-idp')
servicediscovery = boto3.client('servicediscovery')

logger = logging.getLogger()
logger.setLevel(logging.INFO)

def send_email(to_email, subject, html_content, text_content):
    """
    Send a default email to the address. Pull pinpoint app ID and from address from env.
    More information about this service:
    https://docs.aws.amazon.com/pinpoint/latest/developerguide/send-messages-email.html
    Character set is UTF-8.
    Args:
        to_email: Email to send to
        subject: Subject of email
        html_content: HTML version of email content
        text_content: Plain text version of email content
    """

    pinpoint_app_id = os.environ['PinpointAppId']
    response = pinpoint.send_messages(
        ApplicationId=pinpoint_app_id,
        MessageRequest={
            'Addresses': {
                to_email: {
                    'ChannelType': 'EMAIL'
                }
            },
            'MessageConfiguration': {
                'EmailMessage': {
                    'SimpleEmail': {
                        'Subject': {
                            'Charset': "UTF-8",
                            'Data': subject
                        },
                        'HtmlPart': {
                            'Charset': "UTF-8",
                            'Data': html_content
                        },
                        'TextPart': {
                            'Charset': "UTF-8",
                            'Data': text_content
                        }
                    }
                }
            }
        }
    )
    logger.info(f'Message sent to {to_email} and response: {response}')



def send_pickup_email_minimal(to_email):
    """
    Send it to customer saying order ready for pickup as email
    Args:
        to_email: Where to send the email to
    """

    # Specify content:
    subject = "Come pick up your order nearby!"
    heading = "You can pick up your order nearby!"
    intro_text = """
    Welcome, 
    We are waiting for you at Level 3, Door 2 of your Local Retail Demo Store, and Steve from our team will be greeting you with your orders.
    Thank you for shopping!"""
    html_intro_text = intro_text.replace('\n', '</p><p>')

    # Build HTML message
    html = f"""
    <head></head>
    <body>
        <h1>{heading}</h1>
        <p>{html_intro_text}
    </body>
    """

    # Build text message
    text = f"""
{heading}
{intro_text}
    """

    logger.info(f"Contents of email to {to_email} html: \n{html}")
    logger.info(f"Contents of email to {to_email} text: \n{text}")
    send_email(to_email, subject, html, text)

    
def get_order_details(shopper_user_id):
    """
    Grab internal orders service URL from service discovery 
    and get order details from the service for the given user.
    **For this to work, your Lambda needs to be set up on the VPC for the orders service.**
    See: https://docs.aws.amazon.com/lambda/latest/dg/configuration-vpc.html
    Args:
        shopper_user_id: User to get orders for.
    """

    # Get the URL of the carts service
    response = servicediscovery.discover_instances(
        NamespaceName='retaildemostore.local',
        ServiceName='orders',
        MaxResults=1,
        HealthStatus='HEALTHY'
    )
    orders_service_url = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
    logging.info(f'Orders Service Instance IP: {orders_service_url}')

    # Hit orders service to decide if there are waiting orders.
    shopper_user_name = 'user' + shopper_user_id
    orders_url = f'http://{orders_service_url}/orders/username/{shopper_user_name}'
    logging.info(f'Hitting {orders_url}')
    try:        
        # To make this work properly, you need to make sure that the
        # Lambda is able to acces this IP - this is beyond the scope of this notebook.
        # See: https://docs.aws.amazon.com/lambda/latest/dg/configuration-vpc.html
        response = urllib3.PoolManager().request('GET', orders_url, retries=False)
        orders = json.loads(response.data)
    except Exception as e:
        logger.error('Could not get orders - assuming there is none. Exception: ')
        logger.error(e)
        orders=[]
        
    return orders
    

def lambda_handler(event, context):
    """Handle a Location Geofence enter event. Send an email for this user if they have some orders.
        Args:
        event: From EventBridge - contains information from Location
        context: We do not make use of this.
    """
    logger.info('Environment:')
    logger.info(os.environ)
    logger.info('Event:')
    logger.info(event)

    if event['detail-type'] == "Location Geofence Event" and event['detail']['EventType'] == "ENTER":
        location_device_id = event['detail']['DeviceId']
        # We assume that the location device ID is just the cognito ID - see where we insert location changes below.
        cognito_user_id = location_device_id
        
        # Get email and shopper ID from Cognito profile
        user_pool_id = os.environ['UserPoolId']
        response = cognito_idp.admin_get_user(
            UserPoolId=user_pool_id,
            Username=cognito_user_id
        )
        user_attributes = {att['Name']: att['Value'] for att in response['UserAttributes']}
        to_email = user_attributes['email']
        shopper_user_id = user_attributes['custom:profile_user_id']
   
        orders= ["An order!"]  # Let us assume that the customer has an order so we can get an email every time
        logging.info(f'Orders: {json.dumps(orders, indent=2)}')

        if len(orders) > 0:
            # There are waiting orders! Email about them.
            logger.info(f"User {shopper_user_id} has some orders - send her/him an email")
            send_pickup_email_minimal(to_email)
        else:
            logger.info(f"User {shopper_user_id} has no orders")
    
        logger.warning('END')
                        


Let us ensure that that got written where we expect (for those who may not be familiar with ``%%writefile``).

In [ ]:
!tail location-event-handler/location-event-handler.py

If our Lambda function makes use of a non-core module we could install it as follows:

    !pip install --target ./location-event-handler MODULE

Bundle all of that up into a Lambda function.

In [ ]:
! [ -f location-event-handler.zip ] && rm location-event-handler.zip  # remove deployment bundle if exists
!cd location-event-handler && zip -r9 ../location-event-handler.zip .

Create the actual Lambda function in the cloud.

In [ ]:
create_function_response = awslambda.create_function(
        FunctionName='LocationNotebookEventHandler',
        Runtime="python3.8",
        Role=sagemaker_role,
        Handler="location-event-handler.lambda_handler",
        Code={'ZipFile': open('./location-event-handler.zip', 'rb').read()},
        Timeout=20,
        Environment = {'Variables':{
            'UserPoolId': UserPoolId,
            'PinpointAppId': PinpointAppId
        }}
)

function_arn = create_function_response['FunctionArn']

## Connect Location Geofence events up to Lambda

We are going to use the following event pattern with EventBridge to connect up Location with our processing Lambda. For more details about such event patterns see https://docs.aws.amazon.com/eventbridge/latest/userguide/eventbridge-and-event-patterns.html 

In [ ]:

event_pattern = \
     {
        "source": [
            "aws.geo"
        ],
        "resources": [
            geofence_collection_arn
        ],
        "detail-type":[
           "Location Geofence Event"
        ],
        "detail":{
           "EventType": ["ENTER"]
        }
      }
print('Event pattern:\n', json.dumps(event_pattern, indent=4))



We put the event rule and add the Lambda function as its target:

In [ ]:
put_rule_response = events.put_rule(Name=eventbridge_rule_name,
                                    EventPattern=json.dumps(event_pattern))
print(json.dumps(put_rule_response, indent=4))

In [ ]:
put_targets_response = events.put_targets(Rule=eventbridge_rule_name,
                                         Targets=[{'Id': eventbridge_target_id,
                                                   'Arn': function_arn}])
print(json.dumps(put_targets_response, indent=4))

Add a permission to the Lambda function to allow the event rule we just created to invoke the function:

In [ ]:
add_lambda_permission_response = awslambda.add_permission(
    FunctionName='LocationNotebookEventHandler', 
    StatementId='EventBridgeInvokePermission',
    Action="lambda:InvokeFunction", 
    Principal="events.amazonaws.com",
    SourceArn=put_rule_response['RuleArn'],
)
print(json.dumps(add_lambda_permission_response, indent=4))

# Track a device
Let us simulate a customer entering and exiting the geofence so that we get the Geofence event firing, and our custom Lambda above firing. If you fail to get the emails, you can try visiting the Lambda function in the UI console ( https://console.aws.amazon.com/lambda/home#/functions/LocationNotebookEventHandler?tab=monitoring ), clicking on Monitoring, and checking the logs.

The following route starts outside and enters the geofence.

In [ ]:
customer_route = [(i*0.1, 0) for i in range(30, -1, -1)]

In [ ]:
for position in customer_route:
    # Use the UTC time here - important! Otherwise you will be putting future events and not trigger
    t = datetime.utcnow()  
    time.sleep(2) # let us leave two seconds between each step
    print(f"Updating position: {position[0]:0.1f}, {position[1]:0.1f}")
    location.batch_update_device_position(
        TrackerName=resource_name,
        Updates=[{
            'DeviceId': cognito_username,
            'Position': position,
            'SampleTime': t
          }])
    

If your Pinpoint has been set up with a verified email, you should receive a message like this:

![images/retaildemostore-locationpinpoint-email.png](images/retaildemostore-locationpinpoint-email.png)

Yay!

If not, you can try visiting the Lambda function in the UI console ( https://console.aws.amazon.com/lambda/home#/functions/LocationNotebookEventHandler?tab=monitoring ), clicking on Monitoring, and checking the logs.

Let us retrieve the latest position you provided for your "DeviceId" (in this Notebook we choose your Cognito username so that you get emails sent to you):

In [ ]:
location.get_device_position(DeviceId=cognito_username, TrackerName=resource_name)

Or, you can check that the history you provided like this:

In [ ]:
for update in location.get_device_position_history(DeviceId=cognito_username, TrackerName=resource_name)['DevicePositions']:
    print(f"{update['SampleTime'].isoformat()} -- {update['Position'][0]:0.1f}, {update['Position'][1]:0.1f}")

Note that Location's time resolution of update is such that not every update to position will be recorded here.

Let us do a plot of that path against the geofence, so we can visualise the whole thing:

In [ ]:
plt.figure(figsize=[6,5])
for geofence in location.list_geofences(CollectionName=resource_name)['Entries']:
    for polygon in geofence['Geometry']['Polygon']:
        geox = [pos[0] for pos in polygon]
        geoy = [pos[1] for pos in polygon]
        plt.plot(geox, geoy, 'r-', label='Geofence')
device_positions = location.get_device_position_history(DeviceId=cognito_username,TrackerName=resource_name)['DevicePositions']
for ind, update in enumerate(device_positions):
    label = 'Location History' if ind==0 else None
    plt.plot(update['Position'][0], update['Position'][1], 
             color=(1-ind/len(device_positions),ind/len(device_positions),0), 
             linewidth=0, marker='x', label=label)
plt.title(f"Location history for DeviceId '{cognito_username}'")
plt.legend()
plt.show()

# Next steps


The full deployed Location demo version of this workshop explores how Location may be used in many more ways. We suggest you investigate the code and get a feel for how these things are done. 

Additions might include, for example:

 - Full collection handling logic and notifications for stores when users are near to stores.
 - Integration of Location with Amazon Pinpoint marketing "campaigns" by propagating Location events into Pinpoint events. In the full demo we incorporate this into "abandoned cart" campaigns.
 - Further integration with Amazon Personalize by giving localised recommendations. In our full demo we provide coupon offers to users when then get close to the target store.
 - The sky is the limit!

# Location Workshop Cleanup

The remainder of this notebook will walk through deleting all of the resources created. You should only need to perform these steps if you have deployed the Retail Demo Store in your own AWS account and want to deprovision the Location resources. If you are participating in an AWS-led workshop, this process is likely not necessary.

In order, we will delete event rules, lambda function, then location resources.


In [ ]:
print('Detaching from rule', eventbridge_rule_name, 'target', eventbridge_target_id)
remove_targets_response = events.remove_targets(
    Rule=eventbridge_rule_name,
    Ids=[
        eventbridge_target_id
    ]
)

In [ ]:
print('Deleting event rule', eventbridge_rule_name)
delete_rule_response = events.delete_rule(
    Name=eventbridge_rule_name
)

In [ ]:
print('Deleting function LocationNotebookEventHandler')
delete_function_response = awslambda.delete_function(
    FunctionName='LocationNotebookEventHandler'
)

In [ ]:
print('Disassociating tracker', resource_name, 'from geofence', geofence_collection_arn)
disassociate_tracker_consumer_response = location.disassociate_tracker_consumer(
    ConsumerArn=geofence_collection_arn,
    TrackerName=resource_name
)

In [ ]:
print('Deleting map with name', resource_name)
delete_map_response = location.delete_map(MapName=resource_name)

In [ ]:
print('Deleting geofence collection with name', resource_name)
delete_geofence_collection_response = location.delete_geofence_collection(CollectionName=resource_name)

In [ ]:
print('Deleting tracker with name', resource_name)
delete_tracker_response = location.delete_tracker(TrackerName=resource_name)

Done! Have a nice day!